# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
from cassandra.cluster import Cluster
from cassandra.query import dict_factory
import os
import glob
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)
#for count, file in enumerate(file_path_list):
#    print(f'{count+1}. {file}')

/Users/tomaszkowalski/Documents/LearningNewThings/Data-Engineering-with-AWS/data_modeling_project_1


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

8056


In [4]:
# check the number of rows in your csv file
#with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
#    print(sum(1 for line in f))
print(len(pd.read_csv('event_datafile_new.csv')))

6820


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
# Connected to local docker container with default address set to (0.0.0.0)
cluster = Cluster(['0.0.0.0'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()
# Set output query output to dict instead of named tuple
session.row_factory = dict_factory

#### Create Keyspace

In [6]:
try:
    # Create a Keyspace 
    session.execute("""
                    CREATE KEYSPACE IF NOT EXISTS sparkify 
                    WITH REPLICATION = {'class' : 'SimpleStrategy','replication_factor' : 1};
                    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace('sparkify')

### Explore pre-processed data

In [8]:
df = pd.read_csv('event_datafile_new.csv')
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Harmonia,Ryan,M,0,Smith,655.77751,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Sehr kosmisch,26
1,The Prodigy,Ryan,M,1,Smith,260.07465,free,"San Jose-Sunnyvale-Santa Clara, CA",583,The Big Gundown,26
2,Train,Ryan,M,2,Smith,205.45261,free,"San Jose-Sunnyvale-Santa Clara, CA",583,Marry Me,26
3,Sony Wonder,Samuel,M,0,Gonzalez,218.06975,free,"Houston-The Woodlands-Sugar Land, TX",597,Blackbird,61
4,Van Halen,Tegan,F,2,Levine,289.38404,paid,"Portland-South Portland, ME",602,Best Of Both Worlds (Remastered Album Version),80


In [9]:
#df.loc[(df['sessionId'] == 338) & (df['itemInSession'] == 4)]
#df.loc[df['song'] == 'All Hands Against His Own']

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [10]:
# Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
# sessionId = 338, and itemInSession = 4

#The table **music_sessions**,
# the **sessionId** column was used as a PARTITION KEY
# to filter the result by this column. 
# We have used **itemInSession** column as clustering columns to sort the result.

create_tb_query_1 = """
                    CREATE TABLE IF NOT EXISTS music_sessions (
                    artist TEXT, song TEXT, length DECIMAL, session_id INT, itemln_session INT, 
                    PRIMARY KEY (session_id, itemln_session)
                    );
                    """

# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
# for userid = 10, sessionid = 182

create_tb_query_2 = """
                    CREATE TABLE IF NOT EXISTS user_song_sessions (
                    artist TEXT, song TEXT, item_in_session INT, first_name TEXT, last_name TEXT, user_id INT, session_id INT,
                    PRIMARY KEY ((user_id, session_id), item_in_session))
                    WITH CLUSTERING ORDER BY (item_in_session ASC)
                    """

# Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

create_tb_query_3 = """
                    CREATE TABLE IF NOT EXISTS songs_history (
                    user_id INT, first_name TEXT, last_name TEXT, song TEXT, 
                    PRIMARY KEY ((song), user_id)
                    );
                    """

# Drop tables in order to make sure the new data is inserted correctly
session.execute("DROP TABLE IF EXISTS music_sessions")
session.execute("DROP TABLE IF EXISTS user_song_sessions")
session.execute("DROP TABLE IF EXISTS songs_history")

session.execute(create_tb_query_1)
session.execute(create_tb_query_2)
session.execute(create_tb_query_3)

In [11]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Insert into first table 1
        insert_query_1 = "INSERT INTO music_sessions (artist, song, length, session_id, itemln_session)"
        insert_query_1 = insert_query_1 + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(insert_query_1, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))
        # Insert into second table 2
        insert_query_2 = "INSERT INTO user_song_sessions (artist, song, item_in_session, first_name, last_name, user_id, session_id)"
        insert_query_2 = insert_query_2 + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(insert_query_2, (line[0], line[9], int(line[3]), line[1], line[4], int(line[-1]), int(line[8])))
        # Insert into second table 3
        insert_query_3 = "INSERT INTO songs_history (user_id, first_name, last_name, song)"
        insert_query_3 = insert_query_3 + "VALUES (%s, %s, %s, %s)"
        session.execute(insert_query_3, (int(line[-1]), line[1], line[4], line[-2]))

#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
select_query_1 = """
                SELECT artist, song, length
                FROM music_sessions
                WHERE session_id = 338 AND itemln_session = 4;
                """
rows_1 = session.execute(select_query_1)
df_1 = pd.DataFrame(list(rows_1))
df_1

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.3073


In [13]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
select_query_2 = """
                SELECT artist, song, item_in_session, first_name, last_name
                FROM user_song_sessions
                WHERE user_id = 10 AND session_id = 182;
                """
rows_2 = session.execute(select_query_2)
df_2 = pd.DataFrame(list(rows_2))
df_2

,artist,song,item_in_session,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,0,Sylvie,Cruz
1,Three Drives,Greece 2000,1,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,2,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,3,Sylvie,Cruz


In [14]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
select_query_3 = """
                SELECT first_name, last_name
                FROM songs_history
                WHERE song = 'All Hands Against His Own';
                """
rows_3 = session.execute(select_query_3)
df_3 = pd.DataFrame(list(rows_3))
df_3                    

,first_name,last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [15]:
session.execute("DROP TABLE IF EXISTS music_sessions")
session.execute("DROP TABLE IF EXISTS user_song_sessions")
session.execute("DROP TABLE IF EXISTS songs_history")

### Close the session and cluster connection¶

In [16]:
session.shutdown()
cluster.shutdown()